<div class='alert' style='background-color: #273333; color: #E5E7E8; padding:26px 26px; border-radius:15px; font-size:40px;'><B>Show Up </B> for Health - Predict From Model</div><span style='color: #273333; padding:26px 26px; font-size:11px;'> Powered by <B>AutoNote </B>&<b> Show Up for Health</b> Project</B></span><div style='margin:4px 26px; color:#273333; font-size:17px;'>

In [33]:
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from joblib import dump, load

import tensorflow as tf
from tensorflow.keras.models import load_model
from keras import backend as K
from keras.models import load_model

# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
from showupforhealth.utils import *
from showupforhealth.params import *
from showupforhealth.ml_functions.predict import *

<div class='alert' style='background-color: #273333; color: #E5E7E8; padding:12px 12px; border-radius:15px; font-size:16px;'><B>Enter </B> Prediction Surgery Prefix</div>

In [ ]:
surgery_prefix = input('Input Surgery Prefix: ')

---------

In [78]:
X_temp = streamlit_predict(surgery_prefix)
X_new = X_temp.drop(columns='Patient ID')
pt_id = X_temp[["Patient ID"]]
X_new.shape


=== Preparing Appoitment Data for Prediction =================================
🌤️ Prediction: HPVM - preparing appointment data for weather
👩🏻‍🦰 Appointments: 2732 🧑🏻‍🦰 Unique Patient IDs; 1295
🛜 Requesting forcast from Open-Meteo Weather API 2023-08-21 - 2023-09-11
🔂 Merge weather + appointment data
✅ Successful: return df
👉 Appointment DF in: (2732, 11)
🔀 Disease Register merged with prediction data - (2732, 32)
❌ Drop NaN - Disease Register Merge = 297
⛽️ Fill NaN (0) - No Shows Merge

=== Feature Engineering =============================================================
🔂 Rename Columns
🔂 Drop deseased and deducted
🔂 Columns to Datetime
🔂 Fix Appointment Time
🔂 book_to_app_days
🔂 booked_by_clinician
🔂 Extract Rota Types
🔂 registered_for_months
🔂 Week
🔂 Month
🔂 Day of week
🔂 Convert Cyclical data
🔂 Drop Column no longer needed
🔂 Rows dropped from Rotas other than spec: 192
🔂 Rows from with Negative book_to_app_days: 0
🔂 Drop rows with Sex Unknonw & Indeterminate
🔂 Labelencode Column

(2174, 36)

<div class='alert' style='background-color: #273333; color: #E5E7E8; padding:12px 12px; border-radius:15px; font-size:16px;'><B>Scaler </B> Load & Apply</div>

In [79]:
X_new = sort_df_columns(X_new)
scaler = load(f'{MODEL_OUTPUT}/jan_scaler_17sept23.pkl')

X_new = transform_data(X_new, scaler)
X_new = X_new.astype('float32')

<div class='alert' style='background-color: #273333; color: #E5E7E8; padding:12px 12px; border-radius:15px; font-size:16px;'><B>Predict </B> Load Model</div>

In [80]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

In [81]:
model = load_model(f'{MODEL_OUTPUT}/model_weights_2023-09-16 20-49-02.h5', custom_objects={'f1_score': f1_score})

In [82]:
predictions = model.predict(X_new)

68/68 [==============================] - 0s 477us/step


<div class='alert' style='background-color: #273333; color: #E5E7E8; padding:12px 12px; border-radius:15px; font-size:16px;'><B>Threshold </B> Set below to optimize prediction.</div>

In [99]:
threshold = 0.6

In [100]:
# Sample preprocessing (modify based on your actual preprocessing steps)
# X_new = preprocess_function(new_data)
# For binary classification with a sigmoid activation
class_labels = (predictions > threshold).astype(int)
print(f'No shows Predicted at {threshold} threshold: {class_labels.flatten().tolist().count(0)}')

No shows Predicted at 0.6 threshold: 69


<div class='alert' style='background-color: #273333; color: #E5E7E8; padding:12px 12px; border-radius:15px; font-size:16px;'><B>Inspect Outcome </B> Predicted No Shows</div>

In [101]:
class_labels_list = class_labels.flatten().tolist()
new_col = pd.Series(class_labels_list, name='Model_Prediction')

df = pd.concat([pt_id, new_col], axis=1)

surgery = pd.read_csv(f'{PREDICT_DATA}/original/{surgery_prefix}_Predict.csv')
merged = surgery.merge(df, how='left', on="Patient ID")
merged_short = merged[['Appointment status','Model_Prediction', 'Patient ID']]
filtered_df = merged_short[merged_short['Model_Prediction'] == 0]
print(f'Surgery: {surgery_prefix}')
filtered_df

Surgery: HPVM


,Appointment status,Model_Prediction,Patient ID
8,Finished,0.0,48462081
31,Finished,0.0,60716589
32,Finished,0.0,60716589
214,Finished,0.0,47487882
225,Finished,0.0,47476113
268,Finished,0.0,47349379
296,Finished,0.0,60716589
297,Finished,0.0,60716589
332,Did Not Attend,0.0,46031210
405,Finished,0.0,45160553
